In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [110]:
long_short_returns = pd.read_pickle("long_short_returns.pkl")
ls_returns = pd.DataFrame(long_short_returns)
interest_rates = pd.read_csv("FEDFUNDS.csv", parse_dates=['observation_date'], index_col='observation_date')
gdp = pd.read_csv("GDPC1.csv", parse_dates=['observation_date'], index_col='observation_date')
vix = pd.read_csv("VIXCLS.csv", parse_dates=['observation_date'], index_col='observation_date')
cpi = pd.read_csv("FPCPITOTLZGUSA.csv", parse_dates=["observation_date"], index_col="observation_date")
consumer_sentiment = pd.read_csv("UMCSENT.csv", parse_dates=["observation_date"], index_col="observation_date")
credit_spread = pd.read_csv("BAA10Y.csv", parse_dates=['observation_date'], index_col='observation_date')
TB = pd.read_csv("DTB3.csv", parse_dates=['observation_date'], index_col='observation_date')
LGB = pd.read_csv("IRLTLT01USQ156N.csv", parse_dates=['observation_date'], index_col='observation_date')
industrial_production = pd.read_csv("INDPRO.csv", parse_dates=['observation_date'], index_col='observation_date')
consumption = pd.read_csv("PCE.csv", parse_dates=['observation_date'], index_col='observation_date')
oil_prices = pd.read_csv("WPU0561.csv", parse_dates=['observation_date'], index_col='observation_date')
baa = pd.read_csv("BAA.csv", parse_dates=['observation_date'], index_col='observation_date')



interest_rates.rename(columns={"FEDFUNDS": "Interest_Rate"}, inplace=True)
gdp.rename(columns={"GDPC1": "Growth"}, inplace=True)
vix.rename(columns={"VIXCLS": "VIX"}, inplace=True)
cpi.rename(columns={"FPCPITOTLZGUSA": "Inflation"}, inplace=True)
#cpi = inflation_yearly.resample('M').ffill()
consumer_sentiment.rename(columns={"UMCSENT" : "Consumer_Sentiment"}, inplace=True)
credit_spread.rename(columns={"BAA10Y" : "Credit_Spread"}, inplace=True)
TB.rename(columns={"DTB3" : "Treasury Bill"}, inplace=True)
LGB.rename(columns={"IRLTLT01USQ156N" : "LGB"}, inplace=True)
industrial_production.rename(columns={"INDPRO" : "Industrial_Production"}, inplace=True)
consumption.rename(columns={"PCE" : "Consumption"}, inplace=True)
oil_prices.rename(columns={"WPU0561" : "Oil_Prices"}, inplace=True)
inflation = np.log(cpi).diff().dropna()


def to_month_end(df):
    # Wenn PeriodIndex: in Monatsende-DatetimeIndex umwandeln
    if isinstance(df.index, pd.PeriodIndex):
        df.index = df.index.to_timestamp('M')
    else:
        df.index = pd.to_datetime(df.index).to_period('M').to_timestamp('M')
    return df

ls_returns = to_month_end(ls_returns)
interest_rates = to_month_end(interest_rates)
gdp = to_month_end(gdp)
vix = to_month_end(vix)
vix = vix.groupby(vix.index).mean()
consumer_sentiment = to_month_end(consumer_sentiment)
TB = to_month_end(TB)
LGB = to_month_end(LGB)
industrial_production = to_month_end(industrial_production)
consumption = to_month_end(consumption)
oil_prices = to_month_end(oil_prices)
baa = to_month_end(baa)
#inflation = to_month_end(inflation)

#Derived Series
MP = np.log(industrial_production) - np.log(industrial_production.shift(1))
YP = np.log(industrial_production) - np.log(industrial_production.shift(12))
RHO = TB.shift(1) - inflation
DEI = expected_inflation.shift(-1) - expected_inflation
UPR = baa - LGB
UTS = LGB - TB.shift(1)

# Deine Variablen (MP, YP, etc.) definieren

# Indizes ohne Duplikate (wie du schon machst)
RHO = RHO[~RHO.index.duplicated(keep='first')]
UTS = UTS[~UTS.index.duplicated(keep='first')]

# Gemeinsamen Index bestimmen, bei dem keine NaNs auftreten
common_index = MP.dropna().index
common_index = common_index.intersection(YP.dropna().index)
common_index = common_index.intersection(RHO.dropna().index)
common_index = common_index.intersection(DEI.dropna().index)
common_index = common_index.intersection(UPR.dropna().index)
common_index = common_index.intersection(UTS.dropna().index)

# Zusammenführen nur auf gemeinsamen Index
#macros = pd.concat([df.loc[common_index] for df in [MP, YP, RHO, DEI, UPR, UTS]], axis=1)
#macros.columns = ['MP', 'YP', 'RHO', 'DEI', 'UPR', 'UTS']

#ls_returns.index = ls_returns.index.to_timestamp()
ls_returns.index = pd.to_datetime(ls_returns.index)
ls_returns.index = ls_returns.index.normalize()
#macros.index = macros.index.normalize()
#macros.index = macros.index.to_timestamp()
#common_index = ls_returns.index.intersection(macros.index)
#ls_returns = ls_returns.loc[common_index]
#macros = macros.loc[common_index]
#macros

/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x105231330>>
Traceback (most recent call last):
  File "/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [111]:
cpi.tail()

,Inflation
observation_date,
2020-01-01,1.233584
2021-01-01,4.697859
2022-01-01,8.002800
2023-01-01,4.116338
2024-01-01,2.949525


In [55]:
#Conputing unexpected inflation Part 1

# 1. Berechne log Inflation
log_cpi = np.log(cpi)
pi_t = log_cpi.diff().dropna()  # π_t = log(CPI_t) - log(CPI_{t-1})

# 2. Veränderung der Inflation: Δπ_t = π_t - π_{t-1}
delta_pi = pi_t.diff().dropna()

# 3. MA(1)-Modell auf Δπ_t
model = sm.tsa.ARIMA(delta_pi, order=(0, 0, 1))
results = model.fit()

print(results.summary())

/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecastin

                               SARIMAX Results                                
Dep. Variable:              Inflation   No. Observations:                  754
Model:                 ARIMA(0, 0, 1)   Log Likelihood                 264.406
Date:                Sun, 25 May 2025   AIC                           -522.812
Time:                        15:37:39   BIC                           -508.936
Sample:                             0   HQIC                          -517.467
                                - 754                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7.664e-06    2.8e-05     -0.274      0.784   -6.25e-05    4.72e-05
ma.L1         -0.9998      0.074    -13.601      0.000      -1.144      -0.856
sigma2         0.0288      0.002     13.768      0.0

In [61]:
#Computing unexpected inflaiton part 2
theta = results.params['ma.L1']
residuals = results.resid

# Erwartete Inflation: Et[I_t | t-1] = I_{t-1} - θ * ε_{t-1}
expected_inflation = inflation.shift(1) - theta * residuals.shift(1)
unexpected_inflation = inflation - expected_inflation
expected_inflation = expected_inflation.dropna()


In [65]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 1. CPI log differenzieren
inflation = np.log(cpi).diff()

# 2. NaNs entfernen
inflation_clean = inflation.dropna()

# 3. Fit ARIMA(0,0,1)
model = sm.tsa.ARIMA(inflation_clean, order=(0,0,1))
results = model.fit()

theta = results.params['ma.L1']
residuals = results.resid

# 4. Sicherstellen, dass es Series sind
inflation_clean = inflation_clean.squeeze()
residuals = residuals.squeeze()

# 5. Erwartete Inflation berechnen
expected_inflation = inflation_clean.shift(1) - theta * residuals.shift(1)

# 6. Ergebnis prüfen
expected_inflation


/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecastin

observation_date
1960-02-29             NaN
1960-03-31   -3.720485e-07
1960-04-30   -3.721158e-07
1960-05-31   -3.721158e-07
1960-06-30   -3.721158e-07
                  ...     
2023-09-30   -3.721158e-07
2023-10-31   -3.721158e-07
2023-11-30   -3.721158e-07
2023-12-31   -3.721158e-07
2024-01-31   -3.721158e-07
Length: 755, dtype: float64

In [ ]:
#PCA
X = macros
#Standardize Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=X.shape[1])
X_pca = pca.fit_transform(X_scaled)

explained_variance_ratio = pca.explained_variance_ratio_
print("Erklärte Varianz pro Hauptkomponente:", explained_variance_ratio)
print("Kumulative erklärte Varianz:", np.cumsum(explained_variance_ratio))

plt.figure(figsize=(8,5))
plt.bar(range(1, len(explained_variance_ratio)+1), explained_variance_ratio, alpha=0.7, label='Einzelne Varianz')
plt.step(range(1, len(explained_variance_ratio)+1), np.cumsum(explained_variance_ratio), where='mid', label='Kumulative Varianz')
plt.xlabel('Hauptkomponenten')
plt.ylabel('Erklärte Varianz')
plt.legend()
plt.show()

n_components = 1
X_pca_reduced = X_pca[:, :n_components]

# Optional: DataFrame mit PCs für weitere Analyse
X_pca_df = pd.DataFrame(X_pca_reduced, index=X.index, columns=[f'PC{i+1}' for i in range(n_components)])


In [328]:
#GLS
Y = ls_returns
#X = macros commented out, due to PCA
X = macros
X = sm.add_constant(X)
X_pca_df = sm.add_constant(X_pca_df)

ols_model = sm.OLS(Y, X).fit(cov_type="HC3")
residuals = ols_model.resid

resid_sq = residuals ** 2 #Squared residuals
var_model = sm.OLS(resid_sq, X).fit(cov_type="HC3") #estimating the residuals
sigma2_hat = var_model.fittedvalues.clip(lower=1e-8) #avoiding 0

sigma = np.diag(sigma2_hat) #constructing Covariance Matrix

gls_model = sm.GLS(Y, X, sigma=sigma).fit(cov_type="HC3")
print(gls_model.summary())

                            GLS Regression Results                            
Dep. Variable:         long_short_ret   R-squared:                       0.029
Model:                            GLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     9.747
Date:                Sun, 25 May 2025   Prob (F-statistic):           6.71e-05
Time:                        12:04:37   Log-Likelihood:                 1567.2
No. Observations:                 684   AIC:                            -3128.
Df Residuals:                     681   BIC:                            -3115.
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0045      0.002      2.702

In [252]:


X_pca_df = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data["feature"] = X_pca_df.columns
vif_data["VIF"] = [variance_inflation_factor(X_pca_df.values, i) for i in range(X_pca_df.shape[1])]

print(vif_data)

         feature            VIF
0          const  106064.470778
1  Interest_Rate       2.207133
2         Growth       1.659344
3            VIX       1.394873
4      Inflation       2.254602


In [ ]:
#SAMIMAX Modell - Model Fit
from statsmodels.tsa.statespace.sarimax import SARIMAX

X = macros
Y = ls_returns
#Model fit
ar_order = [0,1,2,3,4,5, 6, 7]
i_order = [0,1,2,3,4,5]
ma_order = [0,1,2,3,4,5]
fit = []
for p in ar_order:
    for d in i_order:
        for q in ma_order:
            try:
                model = SARIMAX(Y, exog=X, order=(p,d,q))  # (p,d,q): AR, differenzieren, MA
                model_fit = model.fit()
                 #fit.append({"likelihood" : model_fit.llf, "P": p, "D" : d, "Q" : q})
                fit.append({"aic" : model_fit.aic, "P": p, "D" : d, "Q" : q})
            except:
                continue
#best_model = max(fit, key=lambda x: x["likelihood"])

best_model = min(fit, key=lambda x: x["aic"])
print("Bestes Modell:")
print(best_model)

In [316]:
#SARIMAX 
Y = ls_returns
X = macros
X = sm.add_constant(X)
model = SARIMAX(Y, exog=X, order=(1,0,0))
model_fit = model.fit(cov_type="robust")
print(model_fit.summary())


                               SARIMAX Results                                
Dep. Variable:         long_short_ret   No. Observations:                  274
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 603.057
Date:                Fri, 23 May 2025   AIC                          -1196.113
Time:                        16:31:59   BIC                          -1178.048
Sample:                             0   HQIC                         -1188.862
                                - 274                                         
Covariance Type:               robust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0144      0.012      1.242      0.214      -0.008       0.037
Inflation        -0.0016      0.002     -0.957      0.339      -0.005       0.002
Credit_Spread    -0.0028      0.004     

/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Since all Variance Inflation Factors are well below 10 there is no multicollinearity to be suspected.